<h1 style='font-size:40px'> Evaluation</h1>

<h2 style='font-size:30px'>Model Evaluation & Selection</h2>
<div>
    <ul style='font-size:20px'> 
        <li>
            Até agora, o único parâmetro utilizado para avaliar a qualidade dos nossos modelos criados foi o <em>score</em>, que retorna a porcentagem de acertos na adivinhação da categorias ou números. 
        </li>
        <li>
            No entanto, ele pode ser, em muitas situações, insuficiente para nós avaliarmos a real qualidade dos algoritmos criados. Portanto, esta semana, focaremos em outras métricas qualitativas que nos darão uma compreensão mais completa sobre o quão bom um determinado modelo é.
        </li>
    </ul>
</div>
<div>
    <h3 style='font-size:30px;font-style:italic'> Por que accuracy é insuficiente?</h3>
    <h4 style='font-size:25px;font-style:italic;text-decoration:underline'>Datasets desbalanceados </h4>
    <div>
        <ul style='font-size:20px'>
            <li>
                Suponha que tivéssemos que montar um algoritmo de anúncio de produtos em um site de e-commerce. Sua tarefa deve ser identificar, para cada consumidor, os produtos que não são de seu interesse, de modo a evitar que eles sejam mostrados em propagandas online.
            </li>
            <li>
                Como a página possui milhares de produtos, é razoável pensar que apenas uma minúscula fração deles será, de fato, de interesse ao cliente. Se tivéssemos um dataset com 999 mercadorias que não geram apelo (N) e apenas 1 apelativa (A), um simples sorteador aleatório descobrirá com 99,9% de precisão os produtos desinteressantes.
            </li>
            <li>
                Dessa maneira, montar um modelo de ML com o mesmo score de 99,9% não é nada impressionador. Afinal de contas, um sorteador desprovido de inteligência é capaz de ter o mesmo desempenho.
            </li>
        </ul>
    </div>
</div>

In [1]:
# O dataset é bem simples: Quantas vezes os números (0 a 9) aparecem em dataset.target?
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
dataset = load_digits()
X, y = dataset.data, dataset.target
pd.DataFrame({'Occurences':np.bincount(dataset.target)})

,Occurences
0,178
1,182
2,177
3,183
4,181
5,182
6,181
7,179
8,174
9,180


In [2]:
# Vamos fazer com que esse dataset se torne desbalanceado: contaremos as ocorrências do número 1 versus as ocorrências de números que 
# não são 1 (agora rotulados como 0).
unbalanced_y =y.copy()
unbalanced_y[unbalanced_y!=1] = 0
unbalanced_y

array([0, 1, 0, ..., 0, 0, 0])

In [3]:
# Agora, vamos criar um modelo de Supprt Vector Machines para prever se o número será 1 ou 0.
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(X, unbalanced_y, random_state=0)
svc = SVC(kernel='rbf', C=1, gamma='auto').fit(X_train, y_train)
# Veja como o algoritmo é preciso.
svc.score(X_test, y_test)

0.9088888888888889

In [4]:
# Agora, usaremos um DummyClassfier, que rotula instâncias do dataset de treino de acordo com uma norma.
# Neste caso, ele rotulará todas as instâncias com o nome da categoria com o maior número de ocorrências no dataset de treino.
from sklearn.dummy import DummyClassifier

dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
# E vajam só: o score é o mesmo de nosso SVM. Portanto, ele não tem nada de especial.
dummy_majority.score(X_test, y_test)

0.9044444444444445

In [5]:
! mv /Users/felipeveiga/Desktop/Screen\ Shot\ 2022-03-20\ at\ 08.15.52.png ./dummy2.png

mv: /Users/felipeveiga/Desktop/Screen Shot 2022-03-20 at 08.15.52.png: No such file or directory


<center>
    <h1 style='font-size:30px'>Algumas das estratégias mais utilizadas com o DummyClassifier</h1>
    <img src='dummy1.png'>
    <h1 style='font-size:30px'>Estratégias populares para o DummyRegressor</h1>
    <img src='dummy2.png'>
</center>

<h3 style='font-size:30px;font-style:italic'>Confusion Matrices</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            As matrizes de confusão são um outro método de avaliação de modelos de ML responsáveis por criar um resumo visual da performance dos nossos modelos de ML de acordo com o número de previsões acertadas ou erradas.
        </li>
        <li>
            Ao expor o número de Falsos Positivos ou Negativos, elas já são capazes de oferecer uma informação muito mais rica do que o simples <em> accuracy</em> do algoritmo.
        </li>
    </ul>
</div>

<div>
    <center>
        <h1 style='font-size:30px'> Estrutura da Matriz de Confusão (Classificação Binária)</h1>
        <img src='dummy3.png'>
    </center>
</div>

In [6]:
# Vamos criar uma matriz de confusão baseada no nosso classificador Dummy.
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
y_dummy = dummy.predict(X_test)
# Ou seja, temos 407 negativos verdadeiros, mas 43 negativos falsos.
confusion_matrix(y_test, y_dummy)

array([[407,   0],
       [ 43,   0]])

In [7]:
# Faremos agora uma matriz de de confusão para um Support Vector Machines.
from sklearn.svm import SVC
svc = SVC(kernel='rbf', C=1, gamma='auto').fit(X_train, y_train)
y_svc = svc.predict(X_test)
# Observe como o desempenho é praticamente o mesmo do DummyClassifier.
confusion_matrix(y_test, y_svc)

array([[407,   0],
       [ 41,   2]])

<div>
<hr>
<h2 style='font-size:30px'>Confusion Matrices & Basic Evaluation Metrics</h2>
</div>

<h3 style='font-size:30px;font-style:italic'> Classification Error</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            Consiste basicamente no grau de erros cometidos pelo modelo (1-Accuracy).
        </li>
    </ul>
</div>
<h3 style='font-size:30px;font-style:italic'> True Positive Rate (TPR), ou Recall</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            Quantas instâncias rotuladas como positivas em y_test foram corretamente classificadas pelo modelo (TP / (TP + FN))?
        </li>
    </ul>
</div>
<h3 style='font-size:30px;font-style:italic'> Precision</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            Qual o grau de acerto na classificação das instâncias como positivas (TP + (TP + FP))?
        </li>
    </ul>
</div>
<h3 style='font-size:30px;font-style:italic'> False Positive Rate (FPR), ou Specificity</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            De todas as instâncias classificadas como Positivas, qual foi a taxa de imprecisão (FP / (TP + FP))?
        </li>
    </ul>
</div>

<p style='font-size:15px'> <u>Obs</u>: Às vezes, aumentar o grau de uma dessas métricas implicará na redução de outra! Pense: o que é pior? Haver falsos positivos ou negativos? </p>

<h3 style='font-size:30px;font-style:italic'> F-Score</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            É tido como uma média ponderada entre o <em>Precision</em> e o <em>Recall</em> de um modelo. 
        </li>
    </ul>
</div>
<center>
    <img src='eval2.png'>
</center>
<div>
<ul style='font-size:20px'>
    <li>
        Um beta abaixo de zero dá um peso maior ao valor de <em>Precision</em>. Valores acima de 1 focam em <em>Recall</em>.
    </li>
    <li>
        Quando definimos beta como 1, damos importâncias iguais às duas métricas, caso conhecido como F1-Score.
    </li>
</ul>
</div>

<center>
    <h1 style='font-size:30px'>Recall X Precision</h1>
    <img src='eval1.png'>
</center>

In [5]:
# Vamos calcular o valor dessas métricas para o SVC feito na última aula.
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.svm import SVC
dataset = load_digits()
X, y = dataset.data, dataset.target
unbalanced_y =y.copy()
unbalanced_y[unbalanced_y!=1] = 0
X_train, X_test, y_train, y_test = train_test_split(X, unbalanced_y, random_state=0)
svc = SVC(kernel='rbf', C=1, gamma='auto').fit(X_train, y_train)
y_svc = svc.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_svc)}')
print(f'Precision: {precision_score(y_test, y_svc)}')
print(f'Recall: {recall_score(y_test, y_svc)}')
print(f'F1-Score: {accuracy_score(y_test, y_svc)}')

Accuracy: 0.9088888888888889
Precision: 1.0
Recall: 0.046511627906976744
F1-Score: 0.9088888888888889


In [6]:
# O sklearn possui também um método que produz uma pequena tabela com todas essa métricas juntas.
# Recomenda-se utilizar o comando 'print' para se ter um output melhor.
from sklearn.metrics import classification_report
print(classification_report(y_test, y_svc, target_names=['not 1', '1']))

              precision    recall  f1-score   support

       not 1       0.91      1.00      0.95       407
           1       1.00      0.05      0.09        43

    accuracy                           0.91       450
   macro avg       0.95      0.52      0.52       450
weighted avg       0.92      0.91      0.87       450



<div>
    <hr>
    <h2 style='font-size:30px'> Classifier Decision Functions</h2>
</div>

<ul style='font-size:20px'>
    <li>
        Agora, nos atentaremos a outros outputs que podemos receber de nossos modelos.
    </li>
</ul>

<div>
    <h3 style='font-size:30px;font-style:italic'>predict_proba</h3>
    <div>
        <ul style='font-size:20px'>
            <li>
                Nos mostra a probabilidade dada pelo algoritmo de ele ter rotulado uma certa instância corretamente.
            </li>
            <li>
                Por padrão, o classificador deve rotular uma instância com uma classe com mais de 50 % de certeza. Ajustar esse parâmetro pode torná-lo mais flexível ou conservador.
            </li>
            <li>
                É importante considerar que a qualidade do fit do algoritmo pode influenciar os valores das probabildades. Por exemplo, um modelo viciado pode nos retornar uma probabilidade excessivamente otimista de acerto.
            </li>
        </ul>
    </div>
</div>

In [7]:
# Testando o método a partir de uma Regressão Logística.
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression().fit(X_train, y_train)
# Note como as instâncias rotuladas como positivas possuem uma alta probabilidade de terem sido classificadas
# corretamente.
list(zip(y_test[:20],lr.predict_proba(X_test)[:20,1]))

/Users/felipeveiga/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[(0, 1.1105302320885824e-13),
 (0, 3.820916040524526e-09),
 (0, 2.0852765819646708e-13),
 (0, 3.594567305890125e-10),
 (0, 1.4674152884937387e-10),
 (0, 7.4609132990900846e-06),
 (1, 0.9984928037248136),
 (0, 7.197605121943425e-11),
 (0, 1.0905860508911064e-12),
 (0, 2.1018508378451408e-12),
 (0, 1.4518890422547448e-14),
 (0, 1.7157217800652057e-10),
 (0, 1.010423750421555e-11),
 (0, 1.5388710855602776e-06),
 (0, 1.2837950150975798e-06),
 (0, 1.6606767181580613e-06),
 (1, 0.999994873815623),
 (0, 1.1929485122843553e-15),
 (0, 1.7930402009009878e-06),
 (0, 1.4043305986082489e-13)]

<h3 style='font-size:30px;font-style:italic'>decision_function</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            Conseguimos utilizar a pontuação de confiança retornada pelo modelo para definirmos com qual valor desse índice uma dada instância de treino será rotulada como positiva ou negativa.
        </li>
    </ul>
    <center>
        <img src='eval3.png'>
    </center>
    <ul style='font-size:20px'>
        <li>
            Perceba na imagem que, conforme ajustamos o valor de threshold, o <em>recall</em> e a <em>precision</em> do modelo se alteram.
        </li>
    </ul>
</div>

In [8]:
# Por exemplo, poreríamos definir que todas as instâncias cuja pontuação de confiança seja <-20 sejam classificadas como 0; resto deverá
# ser tido como 1.
list(zip(y_test[0:20], lr.decision_function(X_test)[0:20]))

[(0, -29.828768621087857),
 (0, -19.38277563807029),
 (0, -29.198704709248815),
 (0, -21.746427305626764),
 (0, -22.64234838378661),
 (0, -11.805825264149457),
 (1, 6.495995792253849),
 (0, -23.35468767412377),
 (0, -27.544305902806503),
 (0, -26.88820280800429),
 (0, -31.86332580561389),
 (0, -22.486017074677196),
 (0, -25.31806622518694),
 (0, -13.384459932811797),
 (0, -13.565688727208007),
 (0, -13.308283716448114),
 (1, 12.181143835508673),
 (0, -34.362348410912034),
 (0, -13.231596149524053),
 (0, -29.594045461943136)]

<div>
    <hr>
    <h2 style='font-size:30px'> Precision-recall and ROC curves</h2>
</div>

<div>
    <ul style='font-size:20px'>
        <li>
            Nesta aula, lidaremos com a criação de gráficos que nos expõem de uma maneira melhor as relações entre as diferentes métricas qualitativas analisadas durante esta semana.
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'>Precision-Recall Curves</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            Têm a função de mostrar os valores da <em> Precision</em> e do <em> Recall</em> do modelo para diferentes threshold do valor do método <em>decision_function.</em>
        </li>
    </ul>
</div>

In [9]:
%matplotlib notebook
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
precision, recall, thresholds = precision_recall_curve(y_test, lr.decision_function(X_test))
plt.xlim([0.0, 1.01])
plt.ylim([0.0, 1.01])
plt.plot(precision, recall, label='Precision-Recall Curve')
plt.xlabel('Precision', fontsize=16)
plt.ylabel('Recall', fontsize=16)
plt.show()

<IPython.core.display.Javascript object>

<h3 style='font-size:30px;font-style:italic'>ROC Curve</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            É responsável por mostrar a relação entre a taxa de Positivos Verdadeiros e de Positivos Falsos.
        </li>
        <li>
            O ideal para o gráfico seria que ele se aproxime do ponto em que o TPR é 1 e FPR é 0. Isso o faria ter uma inclinação aguda e uma área elevada abaixo de sua linha.
        </li>
    </ul>
</div>

In [10]:
# O método roc_curve nos dará os TPR's, FPR's e os thresholds usados.
# auc calcula a área abaixo da curva.
from sklearn.metrics import roc_curve, auc
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr.decision_function(X_test))
roc_auc_lr = auc(fpr_lr, tpr_lr)

plt.figure(num=7)
plt.xlim([-0.01, 1.00])
plt.ylim([-0.01, 1.01])
plt.plot(fpr_lr, tpr_lr, lw=3, label='LogRegr ROC curve (area = {:0.2f})'.format(roc_auc_lr))
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve (1-of-10 digits classifier)', fontsize=16)
plt.legend(loc='lower right', fontsize=13)
# Esta linha mostra os TPR's e FPR's de um modelo de adivinhação aleatório em um dataset com categorias em quantidades iguais.
plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
plt.axes().set_aspect('equal')
# Note como o gráfico de nossa Regressão Linear é agudo. Por baixo de sua curva, uma vasta área do plano é coberta.
plt.show()

<IPython.core.display.Javascript object>

<ipython-input-10-979b1e2ddae4>:17: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  plt.axes().set_aspect('equal')


<div> 
    <hr>
    <h2 style='font-size:30px'> Multi-Class Evaluation</h2>
</div>

<div>
    <ul style='font-size:20px'>
    <li>
        Agora que lidamos com as métricas qualitativas de classificação binária, vamos nos atentar àquelas destinadas às multi classificações.
    </li>
</ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Confusion Matrices</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            É possível elaborar matrizes de confusão em problemas de classificações múltiplas. Nesse contexto, é interessante representá-las como um mapa de calor.
        </li>
    </ul>
</div>

In [39]:
# Vamos usar uma aplicação de SVC no dataset de dígitos para criarmos uma matriz de confusão não-binária. 
dataset = load_digits()
# Segregando os dados de treino e teste.
X, y = dataset.data, dataset.target
X_train_mc, X_test_mc, y_train_mc, y_test_mc = train_test_split(X, y, random_state=0)

# Gerando o modelo SVC e treinando-o com X_test_mc
svm = SVC(kernel = 'linear', gamma='auto').fit(X_train_mc, y_train_mc)
svm_predicted_mc = svm.predict(X_test_mc)

In [46]:
# Criando a matriz de confusão e transformando-a em um DataFrame do pandas.
from sklearn.metrics import confusion_matrix
confusion_cm = confusion_matrix(y_test_mc, svm_predicted_mc )
df_cm = pd.DataFrame(confusion_cm, index=[i for i in range(10)], columns=[i for i in range(10)])
df_cm 

,0,1,2,3,4,5,6,7,8,9
0,37,0,0,0,0,0,0,0,0,0
1,0,42,0,0,0,0,0,0,1,0
2,0,0,44,0,0,0,0,0,0,0
3,0,0,0,43,0,0,0,0,1,1
4,0,0,0,0,38,0,0,0,0,0
5,0,0,0,0,0,47,0,0,0,1
6,0,1,0,0,0,0,51,0,0,0
7,0,0,0,0,1,0,0,47,0,0
8,0,3,1,0,0,0,0,0,44,0
9,0,0,0,1,0,1,0,0,1,44


In [51]:
# Com o dataFrame criado, é hora de plotar o heatmap pretendido.
# Usaremos o seaborn para isso.
%matplotlib notebook
import seaborn as sns
# Ao setar annot como True, o número de instâncias que cada quadrado possui é revelado.
sns.heatmap(df_cm, annot=True)
plt.ylabel('True label')
plt.xlabel('Predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 23.52222222222222, 'Predicted label')

In [53]:
# Agora, estamos construindo um novo heatmap para um modelo não tão preciso. Veja como ele se torna útil em identificarmos
# onde e como o algoritmo errou.
svm = SVC(kernel = 'rbf', gamma='auto').fit(X_train_mc, y_train_mc)
svm_predicted_mc = svm.predict(X_test_mc)
confusion_mc = confusion_matrix(y_test_mc, svm_predicted_mc)
df_cm = pd.DataFrame(confusion_mc, index = [i for i in range(0,10)],
                  columns = [i for i in range(0,10)])

plt.figure(figsize = (5.5,4))
sns.heatmap(df_cm, annot=True)
plt.title('SVM RBF Kernel \nAccuracy:{0:.3f}'.format(accuracy_score(y_test_mc, 
                                                                    svm_predicted_mc)))
plt.ylabel('True label')
plt.xlabel('Predicted label');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<h3 style='font-size:30px;font-style:italic'> Macro Average</h3>
<div>
    <ul style='font-size:20px'>
        <li>
        A média macro se refere à média artimética dos valores uma certa métrica qualitativa na classificação das diversas classes de um dataset.
        </li>
    </ul>
</div>
<center>
    <img src='multi1.png'>
</center>
<div>
    <ul style='font-size:20px'>
        <li>
            <strong>Há um erro na imagem: está sendo calculado o <em>recall </em>médio de cada classificação, e não a <em>precision</em></strong>
        </li>
        <li>
            Por se tratar da média macro, os recall's terão o mesmo peso no cálculo da métrica, mesmo que as laranjas sejam a fruta majoritária do dataset.
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'>Micro Average</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            Diferentemente da Macro, a média Minor é calculada considerando todas as instâncias, e não cada classe. Dessa forma, o Minor Average Recall se dará pelo <em>recall</em> do modelo (TP / (TP + FN))
        </li>
    </ul>
</div>
<center>
    <img src='multi2.png'>
</center>
<div>
    <ul style='font-size:20px'>
        <li>
            Com essa média, a categoria com mais instâncias é que impactará mais no valor final.
        </li>
    </ul>
</div>

<center>
    <h1>Orientações para o uso de Micro e Macro Average</h1>
    <img src='multi3.png'>
</center>

In [73]:
print('Recall Macro Médio do SVC: {:.2f}'.format(recall_score(y_test_mc, svm_predicted_mc, average='macro')))
print('Recall Micro Médio do SVC: {:.2f}'.format(recall_score(y_test_mc, svm_predicted_mc, average='micro')))

Recall Macro Médio do SVC: 0.50
Recall Micro Médio do SVC: 0.49


<div>
    <hr>
    <h2 style='font-size:30px'> Regression Evaluation</h2>
</div>

<div>
    <ul style='font-size:20px'>
        <li>
            Diferentemente das tarefas de classificação, usualmente, o <em>score</em> dos modelos de regressão é o suficiente para analisá-los qualitativamente.
        </li>
    </ul>
</div>
<center>
    <h1>Algumas outras métricas de avaliação de Regressões</h1>
    <img src='reg1.png'>
    <p style='font-size:16px'> <u>Obs</u>: O median_absolute_error é bom em situações em que temos os chamados outliers no dataset. Ele os ignorará. Já o mean_squared_error confere um peso maior a erros grandes.</p>
</center>

In [88]:
# É interessante utilizar os DummyRegressor's quando estiver fazendo algum modelo de regressão. Eles possuem
# a mesma lógica dos DummyClassifier's.

# Aqui, lidaremos com o dataset sobre diabetes.
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
X, y = load_diabetes(return_X_y = True)
X = X[:, None, 6]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

lm = LinearRegression().fit(X_train, y_train)
# Criando um DummyRegressor que prevê sempre a média ds valores de y_train
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_train, y_train)

y_predict = lm.predict(X_test)
y_predict_dummy_mean = lm_dummy_mean.predict(X_test)

In [95]:
# Vamos testar algumas das métricas de avaliação de modelos de regressão.
from sklearn.metrics import mean_squared_error, r2_score
print(f'Mean Squared Error do modelo: {mean_squared_error(y_test, y_predict):.2f}')
print(f'Mean Squared Error do Dummy: {mean_squared_error(y_test, y_predict_dummy_mean):.2f}')
print(f'R2 Score do modelo: {r2_score(y_test, y_predict):.2f}')
print(f'R2 Score do Dummy: {r2_score(y_test, y_predict_dummy_mean):.2f}')

Mean Squared Error do modelo: 4646.74
Mean Squared Error do Dummy: 4965.13
R2 Score do modelo: 0.06
R2 Score do Dummy: -0.00


In [122]:
import matplotlib.pyplot as plt
# Plotando as funções de regressão do modelo real e do DummyRegressor.
plt.figure()
plt.scatter(X_test, y_test,  color='black')
plt.plot(X_test, y_predict, color='green', linewidth=2, label = 'Linear Regression')
plt.plot(X_test, y_predict_dummy_mean, color='red', linestyle = 'dashed', 
         linewidth=2, label = 'dummy')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<center>
    <h1> Algumas outras estratégias dos Dummy Regressor's</h1>
    <img src='reg2.png'>
</center>

In [137]:
# Abaixo, você poderá encontrar o nome de todas as métricas qualitativas disponíveis para uso no sklearn.
from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

<div>
    <hr>
    <h2 style='font-size:30px'>Model Selection: Optimizing Classifiers for Different Evaluation Metrics</h2>
</div>

<div>
    <ul style='font-size:20px'>
        <li>
            Esta aula terá o enfoque em treinarmos o nosso repertório sobre métricas de avaliação de modelo.
        </li>
        <li>
            Primeiramente, iremos utilizá-las na aplicação da validação cruzada.
        </li>
    </ul>
</div>

In [123]:
# Na semana anterior, utilizávamos o cross_val_score com o seu output padrão, o r2_score do modelo.
# Agora, modificaremos o parâmetro 'scoring' do modelo a fim de sabermos, por exemplo, o 'recall' do algoritmo.
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.datasets import load_digits
# Tornando o modelo binário para o melhor avaliarmos.
dataset = load_digits()
X, y = dataset.data, dataset.target == 1
clf = SVC(kernel='linear', C=1)
print(f'Score do modelo: {cross_val_score(clf,X, y, cv=5)}')
print('Recall do modelo: {}'.format(cross_val_score(clf,X, y, cv=5, scoring='recall')))
print('AUC do modelo: {}'.format(cross_val_score(clf,X, y, cv=5, scoring='roc_auc')))

Score do modelo: [0.91944444 0.98611111 0.97214485 0.97493036 0.96935933]
Recall do modelo: [0.81081081 0.89189189 0.83333333 0.83333333 0.83333333]
AUC do modelo: [0.9641871  0.9976571  0.99372205 0.99699002 0.98675611]


<h3 style='font-size:30px;font-style:italic'>GridSearchCV</h3>
<div>
    <ul style='font-size:20px'>
        <li>
            Esse objeto tem funcionalidade de fazer validações cruzadas sobre o dataset de treino. 
        </li>
        <li>
            Mas além desse papel, o GridSearchCV também é capaz de analisar o valor de um determinado parâmetro do modelo que oferecerá a melhor performance de uma dada métrica qualitativa.
        </li>
    </ul>
</div>

In [14]:
# Vamos verificar qual o valor de gamma oferecerá a maior accuracy do modelo de SVC.
from sklearn.datasets import load_digits
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
dataset = load_digits()
X, y = dataset.data, dataset.target == 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = SVC(kernel='rbf')
# A métrica qualitativa padrão é a accuracy do modelo.
gscv = GridSearchCV(clf, {'gamma':[0.001, 0.01, 0.05, 0.1, 1, 10, 100]})
gscv.fit(X_train, y_train)
y_decision_fn_scores_acc  = gscv.decision_function(X_test)
#gscv.return_train_score

# O melhor score do modelo foi de cerca de 99,9% com 'gamma' = 0.01
print(f'Melhor valor de gamma: {gscv.best_params_}')
print(f'Melhor score obtido: {gscv.best_score_}')
gscv.cv_results_

Melhor valor de gamma: {'gamma': 0.001}
Melhor score obtido: 0.9985157648354676


{'mean_fit_time': array([0.02007937, 0.14417157, 0.21683559, 0.1538281 , 0.11285439,
        0.13610263, 0.10485339]),
 'std_fit_time': array([0.00436407, 0.01142203, 0.0039819 , 0.00619299, 0.00406596,
        0.03476616, 0.00378156]),
 'mean_score_time': array([0.01267672, 0.0867363 , 0.09567041, 0.09361634, 0.08158197,
        0.08305149, 0.08107409]),
 'std_score_time': array([0.00122735, 0.00227075, 0.00859775, 0.00498451, 0.00226432,
        0.00718044, 0.00369919]),
 'param_gamma': masked_array(data=[0.001, 0.01, 0.05, 0.1, 1, 10, 100],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'gamma': 0.001},
  {'gamma': 0.01},
  {'gamma': 0.05},
  {'gamma': 0.1},
  {'gamma': 1},
  {'gamma': 10},
  {'gamma': 100}],
 'split0_test_score': array([0.9962963 , 0.91851852, 0.8962963 , 0.8962963 , 0.8962963 ,
        0.8962963 , 0.8962963 ]),
 'split1_test_score': array([1.        , 0.91851852, 0.8962963 , 0.

In [141]:
# Mas se quiséssemos analisar o valor de uma outra métrica, como o roc_auc?
# Assim como em 'cross_val_score', basta ajustarmos o parâmetro 'scoring'.
gscv = GridSearchCV(clf,{'gamma':[0.001, 0.01, 0.05, 0.1, 1, 10, 100]}, scoring='roc_auc')
gscv.fit(X_train, y_train)
print(f'Melhor valor de gamma: {gscv.best_params_}')
print(f'Melhor roc_auc obtido: {gscv.best_score_}')
print(f'roc_auc do dataset de teste: {roc_auc_score(y_test, y_decision_fn_scores_acc )}')

Melhor valor de gamma: {'gamma': 0.001}
Melhor roc_auc obtido: 1.0
roc_auc do dataset de teste: 0.99982858122393


<center>
    <h1>Impacto do parâmetro <em>scoring</em> nas fronteiras de decisão dos modelos otimizados </h1>
    <img src='grid1.png'>
</center>

<center>
    <h1>Conclusões finais e outras funcionalidades de Machine Learning </h1>
    <img src='grid2.png'>
</center>

In [144]:
10**-2

0.01

In [143]:
! mv /Users/felipeveiga/Desktop/Screen\ Shot\ 2022-03-29\ at\ 13.34.25.png ./grid2.png

<p style='color:red'>Continuar por Model Selection: Optimizing Classifiers for Different Evaluation Metrics(2:50) -> Ele vai começar a falar sobre o GridSerch</p>